In [18]:
import pandas as pd
import os

In [17]:
data_url = "/Users/ohmpatel/Desktop/CME241/RL-book/project/data"

In [78]:
import os
import pandas as pd

def create_wide_dataframe(data_folder: str) -> pd.DataFrame:
    """
    Reads all .txt files in `data_folder`, each containing
    <DTYYYYMMDD>, <TIME>, <CLOSE>, etc. The resulting DataFrame
    has:
      - A 'time_step' column (int) formed by concatenating date/time (YYYYMMDDHHMMSS).
      - One column per currency pair (derived from filename).
      - Each cell is the <CLOSE> value for that (time_step, currency_pair).
    
    Then sorts rows by the first 8 digits of time_step (date),
    and then by the remaining digits (time).
    """

    dfs_for_merge = []

    for filename in os.listdir(data_folder):
        if filename.endswith(".txt"):
            filepath = os.path.join(data_folder, filename)
            df = pd.read_csv(filepath)

            required_cols = {"<DTYYYYMMDD>", "<TIME>", "<CLOSE>"}
            if not required_cols.issubset(df.columns):
                print(f"Warning: Missing required columns in '{filename}'. Skipping.")
                continue

            # 1) Create time_step = int(<DTYYYYMMDD><TIME>)
            #    e.g. '20010102' + '230100' -> '20010102230100'
            df["time_step"] = (df["<DTYYYYMMDD>"].astype(str) + 
                               df["<TIME>"].astype(str)).astype(int)
            
            # 2) Derive currency pair from the filename (strip .txt)
            currency_pair = os.path.splitext(filename)[0]  # 'AUDJPY.txt' -> 'AUDJPY'

            # 3) Create a "mini DataFrame" with time_step as index,
            #    and one column named after currency_pair (holding <CLOSE>)
            mini_df = df[["time_step", "<CLOSE>"]].copy()
            mini_df = mini_df.rename(columns={"<CLOSE>": currency_pair})
            mini_df.set_index("time_step", inplace=True)

            # 4) Collect for merging
            dfs_for_merge.append(mini_df)

    if not dfs_for_merge:
        print("No valid data found.")
        return pd.DataFrame(columns=["time_step"])

    # 5) Merge everything side-by-side on time_step
    df_wide = pd.concat(dfs_for_merge, axis=1)

    # 6) time_step is the index. Move it to a column for sorting.
    df_wide.reset_index(inplace=True)

    # 7) Sort time_step as if it's YYYYMMDDHHMMSS:
    #    - first 8 digits -> date_part
    #    - remaining digits -> time_part
    #    - then sort by (date_part, time_part)
    time_str = df_wide["time_step"].astype(str)
    date_part = time_str.str[:8].astype(int)
    time_part = time_str.str[8:].astype(int)

    df_wide["date_part"] = date_part
    df_wide["time_part"] = time_part

    df_wide.sort_values(by=["date_part", "time_part"], inplace=True, ascending=[True, True])

    # 8) Drop the helper columns after sorting
    df_wide.drop(["date_part", "time_part"], axis=1, inplace=True)

    return df_wide

In [79]:
data = create_wide_dataframe(data_url)

In [80]:
data

,time_step,USDCAD,NZDUSD,GBPCHF,NZDJPY,AUDJPY,GBPJPY,GBPUSD,AUDUSD,EURCAD,XAGUSD,EURGBP,EURJPY,USDCHF,USDJPY,EURCHF,CHFJPY
7899291,20010102230100,NaN,NaN,2.4015,NaN,64.30,171.86,1.5018,0.5617,NaN,NaN,0.6328,108.81,1.5988,NaN,1.5204,71.55
7899292,20010102230200,NaN,NaN,2.4016,NaN,NaN,171.87,1.5019,NaN,NaN,NaN,0.6327,108.79,1.5988,NaN,1.5202,71.55
7899293,20010102230300,NaN,NaN,2.4016,NaN,64.29,171.87,1.5019,NaN,NaN,4.58,0.6327,108.80,1.5991,114.43,1.5203,71.55
7899294,20010102230400,NaN,NaN,2.4019,NaN,64.29,171.88,1.5019,0.5616,NaN,NaN,0.6328,108.81,1.5991,114.44,1.5205,71.54
7899295,20010102230500,NaN,NaN,2.4016,NaN,64.29,171.88,1.5018,NaN,NaN,NaN,0.6328,108.81,1.5990,114.44,1.5204,71.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7837871,20230929185500,1.3567,0.5997,1.1164,89.56,96.07,182.22,1.2201,0.6433,1.4344,22.17,0.8663,157.90,0.9150,149.34,0.9674,163.17
7837872,20230929185600,1.3567,0.5997,1.1164,89.56,96.07,182.22,1.2201,0.6433,1.4344,22.17,0.8663,157.90,0.9150,149.34,0.9674,163.17
7837873,20230929185700,1.3567,0.5997,1.1164,89.56,96.07,182.22,1.2201,0.6433,1.4344,22.17,0.8663,157.90,0.9150,149.34,0.9674,163.17
7837874,20230929185800,1.3567,0.5997,1.1164,89.56,96.07,182.22,1.2201,0.6433,1.4344,22.17,0.8663,157.90,0.9150,149.34,0.9674,163.17


In [81]:
data.to_csv("rl_data.csv")

In [82]:
len(data.index)

8329424

In [46]:
start_times = []
for key in data:
    start_times.append(data[key]["<TIME>"].iloc[0])

In [49]:
start = min(start_times)

In [52]:
for index, row in sample.iterrows():
    row["<TIME>"] = (row["<TIME>"] - start) % 100

In [60]:
time_ns = sample['<TIME>'].view('int64')

/var/folders/__/45vv5xv57jdctpcpjyw3ybdm0000gn/T/ipykernel_63043/2324374609.py:1: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  time_ns = sample['<TIME>'].view('int64')


In [57]:
time_ns = (time_ns - start) % 100

In [61]:
list(time_ns)

[230700,
 231200,
 231400,
 234500,
 234600,
 234900,
 235000,
 235100,
 235200,
 235300,
 235400,
 235500,
 235700,
 300,
 900,
 1100,
 2700,
 2900,
 3000,
 3100,
 3200,
 3300,
 3800,
 4200,
 4300,
 4400,
 4500,
 4600,
 4700,
 4800,
 4900,
 5000,
 5100,
 5200,
 5500,
 5700,
 10000,
 10100,
 10200,
 10300,
 10400,
 10500,
 10600,
 10700,
 10800,
 11000,
 11100,
 11200,
 11300,
 11400,
 11500,
 11700,
 11800,
 11900,
 12000,
 12100,
 12200,
 12300,
 12400,
 12500,
 12900,
 13100,
 14200,
 14700,
 14800,
 14900,
 15200,
 15300,
 15600,
 15800,
 20200,
 20700,
 20900,
 21500,
 21800,
 22200,
 22300,
 22400,
 22600,
 23000,
 23200,
 23600,
 23800,
 24600,
 25000,
 25400,
 25700,
 25800,
 25900,
 30100,
 30300,
 30400,
 30500,
 30600,
 30800,
 30900,
 31000,
 31200,
 31400,
 31900,
 32000,
 32200,
 32300,
 32400,
 32500,
 32700,
 33000,
 33100,
 33600,
 33800,
 33900,
 34500,
 34600,
 34800,
 34900,
 35000,
 35100,
 35200,
 35600,
 40000,
 40100,
 40300,
 40500,
 40800,
 40900,
 41000,
 411